# EXPLORATORY DATA ANALYSIS

In [1]:
%run "dataloader.ipynb"

C:\Users\mikyg\AppData\Local\Temp\ipykernel_11196\367482430.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('../data/train.csv', parse_dates=True, index_col="Date")


In [2]:
#importing the libraries
import numpy as np
import pandas as pd
from scipy import stats
import datetime 
import plotly.express as px
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
import os
sys.path.insert(0, '../scripts/')
import logging
from logger_creator import log

In [4]:
print(f"Total shape: {train_df.shape}")
# set a logger file
logger = log(path="../logs/", file="exploratory_data_analysis_log.log")
logger.info("Starts EDA")

INFO Starts EDA


Total shape: (1017209, 8)


In [5]:
train_df.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
Date,,,,,,,,
2015-07-31,1,5,5263,555,1,1,0,1
2015-07-31,2,5,6064,625,1,1,0,1
2015-07-31,3,5,8314,821,1,1,0,1
2015-07-31,4,5,13995,1498,1,1,0,1
2015-07-31,5,5,4822,559,1,1,0,1


In [6]:
# data extraction
train_df['Year'] = train_df.index.year
train_df['Month'] = train_df.index.month
train_df['Day'] = train_df.index.day
train_df['WeekOfYear'] = train_df.index.weekofyear
# adding new variable
train_df['SalePerCustomer'] = train_df['Sales']/train_df['Customers']
train_df['SalePerCustomer'].describe()

count    844340.000000
mean          9.493619
std           2.197494
min           0.000000
25%           7.895563
50%           9.250000
75%          10.899729
max          64.957854
Name: SalePerCustomer, dtype: float64